In [1]:
# セル1: ライブラリのインポート
import nest_asyncio
import aiohttp
import asyncio
import nest_asyncio
import requests
import csv
from typing import Dict, List, Optional, Callable, Any
import yaml
import logging
from datetime import datetime
import os
from tqdm import tqdm
import time
from time import sleep
import traceback
import dotenv
from pathlib import Path
import random

In [2]:
# セル2: モジュールのエクスポート設定
__all__ = ['AmazonProductAnalyzer']

In [7]:
# セル3: APIRateLimiterクラス定義
class EnhancedAPIRateLimiter:
    def __init__(self, requests_per_second=8.0):
        """
        より洗練されたレート制限管理クラス
        
        Parameters:
        -----------
        requests_per_second : float
            1秒あたりの最大リクエスト数
        """
        self.requests_per_second = requests_per_second
        self.min_interval = 1.0 / requests_per_second
        self.last_request_times = []  # 直近のリクエスト時間を記録
        self.window_size = int(requests_per_second * 2)  # 監視するリクエスト履歴のサイズ
    
    def wait_if_needed(self):
        """
        必要に応じて待機してレート制限に適合させる
        """
        current_time = time.time()
        
        # 古すぎるリクエスト履歴を削除（1秒以上前のもの）
        while self.last_request_times and current_time - self.last_request_times[0] > 1.0:
            self.last_request_times.pop(0)
        
        # 現在の履歴サイズがウィンドウサイズ以上なら待機
        if len(self.last_request_times) >= self.window_size:
            # 最も古いリクエストから1秒経過するまで待機
            oldest_request = self.last_request_times[0]
            wait_time = max(0, 1.0 - (current_time - oldest_request))
            if wait_time > 0:
                time.sleep(wait_time)
                current_time = time.time()  # 待機後の時間を更新
        
        # 直近のリクエスト間隔が最小間隔より小さい場合も待機
        if self.last_request_times and (current_time - self.last_request_times[-1] < self.min_interval):
            wait_time = self.min_interval - (current_time - self.last_request_times[-1])
            time.sleep(wait_time)
            current_time = time.time()  # 待機後の時間を更新
        
        # 現在のリクエスト時間を記録
        self.last_request_times.append(current_time)
        return current_time

In [8]:
# セル4: 基本クラス定義と初期化メソッド
class AmazonProductAnalyzer:
    def __init__(self, config_path: str = None):
        # プロジェクトルートディレクトリの検出
        self.root_dir = self._find_project_root()
        
        # 環境変数の読み込み
        dotenv.load_dotenv(os.path.join(self.root_dir, '.env'))
        
        # ディレクトリパスの設定
        self.data_dir = os.path.join(self.root_dir, 'data')
        self.log_dir = os.path.join(self.root_dir, 'logs')
        
        # ディレクトリが存在しない場合は作成
        os.makedirs(self.data_dir, exist_ok=True)
        os.makedirs(self.log_dir, exist_ok=True)
        
        # configファイルのパスを設定
        if config_path is None:
            config_path = os.path.join(self.root_dir, 'config', 'settings.yaml')
            
        self.setup_logging()
        self.config = self.load_config(config_path)
        
        # 環境変数から認証情報を取得して設定ファイルにマージ
        self._merge_env_variables()
        
        self.access_token = self.get_access_token()
        
        # トークン取得時刻を記録
        self.token_timestamp = time.time()
        
        # 出力設定の初期化（なければデフォルト値を設定）
        if 'output' not in self.config['sp_api']:
            self.config['sp_api']['output'] = {
                'input_file': os.path.join(self.data_dir, 'netsea_scraping.csv'),
                'output_file': os.path.join(self.data_dir, 'sp_api_output.csv')
            }
        else:
            # 相対パスを絶対パスに変換
            for key in ['input_file', 'output_file']:
                if key in self.config['sp_api']['output']:
                    rel_path = self.config['sp_api']['output'][key]
                    if not os.path.isabs(rel_path):
                        self.config['sp_api']['output'][key] = os.path.join(self.data_dir, rel_path)

        # レート制限管理の設定
        requests_per_second = self.config['sp_api'].get('requests_per_second', 2.0)
        self.rate_limiter = EnhancedAPIRateLimiter(requests_per_second)


    def _find_project_root(self):
        """
        プロジェクトのルートディレクトリを検出する
        """
        # 現在のファイルの絶対パスを取得
        current_dir = os.path.abspath(os.getcwd())
        
        # 親ディレクトリを探索
        path = Path(current_dir)
        while True:
            # .gitディレクトリがあればそれをルートとみなす
            if (path / '.git').exists():
                return str(path)
            
            # プロジェクトのルートを示す他のファイル/ディレクトリの存在チェック
            # 例: setup.py, README.md, etc.
            if (path / 'setup.py').exists() or (path / 'README.md').exists():
                return str(path)
            
            # これ以上上の階層がない場合は現在のディレクトリを返す
            if path.parent == path:
                return str(path)
            
            # 親ディレクトリへ
            path = path.parent



    def _merge_env_variables(self):
        """
        環境変数から認証情報を取得し、設定ファイルにマージする
        """
        # SP APIの認証情報
        if 'sp_api' not in self.config:
            self.config['sp_api'] = {}
        
        # 環境変数から認証情報を取得
        env_vars = {
            'client_id': os.getenv('SP_API_CLIENT_ID'),
            'client_secret': os.getenv('SP_API_CLIENT_SECRET'),
            'refresh_token': os.getenv('SP_API_REFRESH_TOKEN'),
            'marketplace_id': os.getenv('SP_API_MARKETPLACE_ID')
        }
        
        # 設定ファイルに環境変数の値をマージ（環境変数が設定されている場合のみ）
        for key, value in env_vars.items():
            if value is not None:
                self.config['sp_api'][key] = value
        
        # リクエスト間隔の設定（デフォルト値）
        if 'request_delay' not in self.config['sp_api']:
            self.config['sp_api']['request_delay'] = 1.0  # デフォルトは1秒
            


    
    # 新規メソッド: トークンの更新が必要かどうかを確認し、必要なら更新する
    def refresh_token_if_needed(self):
        """アクセストークンの有効期限をチェックし、必要に応じて更新する"""
        import time
        
        # 現在の時刻とトークン取得時刻の差を計算（秒）
        current_time = time.time()
        elapsed_time = current_time - self.token_timestamp
        
        # トークンの有効期限（55分=3300秒と設定）
        # 60分ではなく余裕を持たせて55分に設定
        token_lifetime = 3300  # 55分 * 60秒
        
        # 有効期限が近づいていれば更新
        if elapsed_time > token_lifetime:
            logging.info(f"アクセストークンの有効期限が近いため更新します (経過時間: {elapsed_time/60:.1f}分)")
            self.access_token = self.get_access_token()
            self.token_timestamp = time.time()
            logging.info("アクセストークンを更新しました")

    

    def setup_logging(self):
        """ログ設定"""
        log_file = os.path.join(self.log_dir, f'sp_api_{datetime.now().strftime("%Y%m%d_%H%M%S")}.log')
        
        # すでに存在するハンドラをすべて削除（重複を防ぐため）
        logger = logging.getLogger('')
        while logger.handlers:
            logger.removeHandler(logger.handlers[0])
        
        # 基本設定
        logging.basicConfig(
            filename=log_file,
            level=logging.DEBUG,  # INFOレベルを非表示にするためWARNINGに変更
            format='%(asctime)s - %(levelname)s - %(message)s',
            encoding='utf-8'  # エンコーディングを明示的に指定
        )
        
        # コンソールにもログを出力（WARNINGレベル以上のみ）
        console = logging.StreamHandler()
        console.setLevel(logging.WARNING)  # INFOレベルを非表示にするためWARNINGに変更
        console.setFormatter(logging.Formatter('%(asctime)s - %(levelname)s - %(message)s'))
        logging.getLogger('').addHandler(console)
        
        # ログファイルの場所を明示的に表示
        print(f"ログファイル: {log_file}")
        logging.warning(f"ログ機能の初期化が完了しました: {log_file}")  # INFOからWARNINGに変更

        
    def load_config(self, config_path: str) -> dict:
        """設定ファイルの読み込み"""
        try:
            with open(config_path, 'r', encoding='utf-8') as f:
                return yaml.safe_load(f)
        except Exception as e:
            logging.error(f"設定ファイルの読み込みに失敗: {str(e)}")
            raise

In [13]:
# セル5: API認証とトークン管理
class AmazonProductAnalyzer(AmazonProductAnalyzer):
    def get_access_token(self) -> str:
        """アクセストークンの取得"""
        token_url = 'https://api.amazon.com/auth/o2/token'
        token_data = {
            'grant_type': 'refresh_token',
            'refresh_token': self.config['sp_api']['refresh_token'],
            'client_id': self.config['sp_api']['client_id'],
            'client_secret': self.config['sp_api']['client_secret']
        }
        try:
            response = requests.post(token_url, data=token_data).json()
            return response['access_token']
        except Exception as e:
            logging.error(f"アクセストークンの取得に失敗: {str(e)}")
            raise

In [15]:
# セル6: コード判定機能
class AmazonProductAnalyzer(AmazonProductAnalyzer):
    def identify_code_type(self, code: str) -> tuple:
        """
        入力コードがJAN/EANかASINかを判定し、必要に応じて形式を修正する
        
        Returns:
        --------
        tuple: (code_type, normalized_code)
            code_type: コードタイプ（'EAN'または'ASIN'）
            normalized_code: 正規化されたコード（EANの場合は13桁になるよう0埋め）
        """
        code = str(code).strip()
        
        # JANコードまたはEANコードの判定（数字のみで構成されている）
        if code.isdigit():
            # 既に13桁の場合はそのまま
            if len(code) == 13:
                return 'EAN', code
            # 12桁以下の場合は先頭に0を追加して13桁にする
            elif 5 <= len(code) <= 12:
                normalized_code = code.zfill(13)  # 0埋めして13桁にする
                print(f"コードを正規化: {code} → {normalized_code} (13桁EAN)")
                return 'EAN', normalized_code
        
        # ASINの判定（10桁の英数字 かつ 最初がB0で始まる）
        if (len(code) == 10 and 
            all(c.isalnum() for c in code) and 
            code.startswith('B0')):
            return 'ASIN', code
        
        # どちらにも当てはまらない場合
        raise ValueError(f"無効なコード形式: {code} - JAN/EANコード(5-13桁の数字)またはASIN(10桁かつB0で始まる英数字)である必要があります") 

In [17]:
# セル7: カタログデータ解析機能
class AmazonProductAnalyzer(AmazonProductAnalyzer):
    def get_item_data_by_code(self, code: str, code_type: str) -> Dict:
        """
        コード（JANまたはASIN）から商品情報を取得
        
        Parameters:
        -----------
        code : str
            コード値
        code_type : str
            コードタイプ ('EAN' または 'ASIN')
            
        Returns:
        --------
        dict or None: 商品情報。取得できない場合はNone
        """
        # トークンの更新チェック
        self.refresh_token_if_needed()
        
        # APIエンドポイント（v2022-04-01）
        url = 'https://sellingpartnerapi-fe.amazon.com/catalog/2022-04-01/items'
        
        headers = {
            'x-amz-access-token': self.access_token,
            'Accept': 'application/json'
        }
        
        # クエリパラメータ
        params = {
            'marketplaceIds': self.config['sp_api']['marketplace_id'],
            'identifiers': code,
            'identifiersType': code_type,
            'includedData': 'attributes,dimensions,identifiers,images,productTypes,relationships,salesRanks,summaries'
        }
        
        logging.info(f"Catalog API v2022-04-01 リクエスト: {code_type} {code}")
        
        # リトライ設定
        MAX_RETRIES = 5
        BASE_DELAY = 2  # 基本待機時間（秒）
        
        for attempt in range(MAX_RETRIES):
            try:
                # レート制限に従って待機
                self.rate_limiter.wait_if_needed()
                
                # APIリクエスト
                response = requests.get(url, headers=headers, params=params)
                
                # レート制限エラーの処理
                if response.status_code == 429:
                    # ヘッダーからリトライ時間を取得（ない場合はバックオフアルゴリズムで計算）
                    retry_after = int(response.headers.get('Retry-After', BASE_DELAY * (2 ** attempt)))
                    logging.warning(f"レート制限に達しました。{retry_after}秒待機します... (試行 {attempt+1}/{MAX_RETRIES})")
                    print(f"⚠️ レート制限に達しました。{retry_after}秒待機します...")
                    time.sleep(retry_after)
                    continue
                
                # トークン期限切れの処理
                if response.status_code == 403:
                    response_text = response.text
                    if "expired" in response_text or "Unauthorized" in response_text:
                        logging.warning("トークン期限切れを検出。トークンを更新します。")
                        self.access_token = self.get_access_token()
                        self.token_timestamp = time.time()
                        headers['x-amz-access-token'] = self.access_token  # ヘッダーを更新
                        time.sleep(BASE_DELAY)
                        continue
                        
                # その他のエラー
                if response.status_code != 200:
                    logging.error(f"Catalog API エラー: {response.status_code} - {response.text}")
                    if attempt < MAX_RETRIES - 1:
                        wait_time = BASE_DELAY * (2 ** attempt)  # 指数バックオフ
                        logging.info(f"リトライ待機中... {wait_time}秒 (試行 {attempt+1}/{MAX_RETRIES})")
                        time.sleep(wait_time)
                        continue
                    return None
                
                # レスポンスの解析
                data = response.json()
                
                # items配列が存在し、要素があるか確認
                if 'items' not in data or len(data['items']) == 0:
                    logging.warning(f"{code_type} {code} の商品情報が見つかりませんでした")
                    return None
                
                # 商品情報の取得（最初の要素を返す）
                item_data = data['items'][0]
                
                # ASIN情報を追加（JAN検索時に必要）
                if code_type == 'EAN' and 'asin' not in item_data:
                    identifiers = item_data.get('identifiers', [])
                    for identifier_set in identifiers:
                        if identifier_set.get('identifierType') == 'ASIN':
                            identifier_values = identifier_set.get('identifiers', [])
                            for id_value in identifier_values:
                                if id_value.get('marketplaceId') == self.config['sp_api']['marketplace_id']:
                                    item_data['asin'] = id_value.get('identifier')
                                    break
                
                logging.info(f"Catalog API v2022-04-01 成功: {code_type} {code}")
                return item_data
                
            except Exception as e:
                logging.error(f"データ取得エラー ({code_type} {code}): {str(e)}")
                logging.error(traceback.format_exc())
                
                if attempt < MAX_RETRIES - 1:
                    wait_time = BASE_DELAY * (2 ** attempt)  # 指数バックオフ
                    logging.info(f"リトライ待機中... {wait_time}秒 (試行 {attempt+1}/{MAX_RETRIES})")
                    time.sleep(wait_time)
        
        return None

    def parse_catalog_data(self, item: Dict) -> Dict:
        """
        Catalog APIのレスポンスから必要な情報を抽出して整形
        
        Parameters:
        -----------
        item : dict
            Catalog APIから取得した商品情報
            
        Returns:
        --------
        dict: 整形された商品基本情報
        """
        # 結果の初期化
        result = {
            '参考価格': None,
            'パッケージ最長辺': None,
            'パッケージ中辺': None,
            'パッケージ最短辺': None,
            'パッケージ重量': None,
            '現在ランキング': None
        }
        
        try:
            # デバッグ用：データ構造の確認
            logging.debug(f"API応答データ構造: {json.dumps(item, indent=2, ensure_ascii=False)[:500]}...")
            
            # 属性データの取得
            attributes = item.get('attributes', {})
            
            # 参考価格の取得 - 構造を修正
            try:
                # list_priceの取得試行
                list_price_attrs = attributes.get('list_price', [])
                
                if list_price_attrs:
                    logging.debug(f"list_price属性: {list_price_attrs}")
                    
                    # デバッグログの形式に合わせて修正: [{'currency': 'JPY', 'value': 1320.0, 'marketplace_id': 'A1VC38T7YXB528'}]
                    for attr in list_price_attrs:
                        if isinstance(attr, dict) and attr.get('marketplace_id') == self.config['sp_api']['marketplace_id']:
                            if 'value' in attr and isinstance(attr['value'], (int, float)):
                                # 実際の構造に合わせて直接valueを使用
                                result['参考価格'] = float(attr['value'])
                                break
            except Exception as e:
                logging.error(f"参考価格の解析エラー: {str(e)}")
            
            # 寸法と重量の属性を安全に取得する関数
            def get_dimension(attr_names):
                """
                複数の可能性のある属性名から寸法データを取得
                
                Parameters:
                -----------
                attr_names : list
                    確認する属性名のリスト
                    
                Returns:
                --------
                tuple: (値, 単位) または (None, None)
                """
                for attr_name in attr_names:
                    try:
                        attrs = attributes.get(attr_name, [])
                        for attr in attrs:
                            if isinstance(attr, dict) and attr.get('marketplace_id') == self.config['sp_api']['marketplace_id']:
                                # データ構造チェック
                                if 'value' in attr and 'unit' in attr:
                                    try:
                                        value = float(attr['value'])
                                        unit = attr['unit'].lower()
                                        return value, unit
                                    except (ValueError, TypeError):
                                        pass
                                
                                # 代替データ構造チェック：valueが辞書の場合
                                elif 'value' in attr and isinstance(attr['value'], dict):
                                    value_dict = attr['value']
                                    if 'amount' in value_dict and 'unit' in value_dict:
                                        try:
                                            value = float(value_dict['amount'])
                                            unit = value_dict['unit'].lower()
                                            return value, unit
                                        except (ValueError, TypeError):
                                            pass
                    except Exception as e:
                        logging.error(f"{attr_name}の解析エラー: {str(e)}")
                return None, None
            
            # パッケージサイズと重量の取得
            # パッケージサイズと重量の取得 - 簡潔なバージョン
            dimensions = {}
            
            try:
                # dimensionsセクションを取得
                dimensions_data = item.get('dimensions', [])
                
                # マーケットプレースIDに一致するデータを探す
                for dim_obj in dimensions_data:
                    if dim_obj.get('marketplaceId') != self.config['sp_api']['marketplace_id']:
                        continue
                        
                    # まずpackageデータを確認し、なければitemデータを使用
                    for container_type in ['package', 'item']:
                        container = dim_obj.get(container_type, {})
                        if not container:
                            continue
                            
                        # 寸法情報（高さ、長さ、幅）を取得
                        for dim_type in ['height', 'length', 'width']:
                            dim_data = container.get(dim_type, {})
                            if dim_data and 'value' in dim_data and 'unit' in dim_data:
                                value = float(dim_data['value'])
                                unit = dim_data['unit'].lower()
                                # インチからcmへの変換
                                if unit in ['inches', 'inch']:
                                    value *= 2.54
                                dimensions[dim_type] = value
                        
                        # 重量情報を取得
                        weight_data = container.get('weight', {})
                        if weight_data and 'value' in weight_data and 'unit' in weight_data:
                            value = float(weight_data['value'])
                            unit = weight_data['unit'].lower()
                            # 単位変換
                            if unit in ['pounds', 'pound', 'lb', 'lbs']:
                                value *= 453.592  # ポンドからグラム
                            elif unit in ['kilograms', 'kg']:
                                value *= 1000  # キログラムからグラム
                            result['パッケージ重量'] = round(value, 2)
                        
                        # 寸法を取得できたら、次のマーケットプレースへ
                        if dimensions:
                            break
                    
                    # 何かしらのデータが取得できたらループを抜ける
                    if dimensions or result['パッケージ重量'] is not None:
                        break
                
                # パッケージサイズの計算（値を降順にソート）
                if dimensions:
                    dim_values = sorted([value for value in dimensions.values()], reverse=True)
                    if len(dim_values) >= 3:
                        result['パッケージ最長辺'] = round(dim_values[0], 2)
                        result['パッケージ中辺'] = round(dim_values[1], 2)
                        result['パッケージ最短辺'] = round(dim_values[2], 2)
                    elif len(dim_values) == 2:
                        result['パッケージ最長辺'] = round(dim_values[0], 2)
                        result['パッケージ中辺'] = round(dim_values[1], 2)
                    elif len(dim_values) == 1:
                        result['パッケージ最長辺'] = round(dim_values[0], 2)
            except Exception as e:
                logging.error(f"パッケージサイズの解析エラー: {str(e)}")
                
            # ランキング情報の取得
            try:
                sales_ranks = item.get('salesRanks', [])
                if sales_ranks:
                    logging.debug(f"salesRanks構造: {sales_ranks}")
                    
                    # デバッグログの実際の構造に合わせて修正
                    for rank_category in sales_ranks:
                        # displayGroupRanksを確認
                        if 'displayGroupRanks' in rank_category:
                            display_ranks = rank_category.get('displayGroupRanks', [])
                            for rank_info in display_ranks:
                                if 'rank' in rank_info:
                                    try:
                                        result['現在ランキング'] = int(rank_info['rank'])
                                        break
                                    except (ValueError, TypeError):
                                        pass
                        
                        if result['現在ランキング'] is not None:
                            break
            except Exception as e:
                logging.error(f"ランキング情報の解析エラー: {str(e)}")
            
            # その他の必要な情報を追加（商品名、メーカー、ブランドなど）
            try:
                # 商品サマリー情報の取得
                summaries = item.get('summaries', [])
                item_info = summaries[0] if summaries else {}
                
                # 商品名
                if '商品名' not in result and item_info.get('itemName'):
                    result['商品名'] = item_info.get('itemName')
                    
                # 商品画像
                if '画像URL' not in result and item_info.get('mainImage', {}).get('link'):
                    result['画像URL'] = item_info.get('mainImage', {}).get('link')
                    
                # ブランド名を複数の属性から取得を試みる
                brand_attrs = attributes.get('brand', [])
                for attr in brand_attrs:
                    if isinstance(attr, dict) and attr.get('marketplace_id') == self.config['sp_api']['marketplace_id']:
                        if 'value' in attr:
                            result['ブランド名'] = attr['value']
                            break
                            
                # メーカー名を複数の属性から取得を試みる
                manufacturer_attrs = attributes.get('manufacturer', [])
                for attr in manufacturer_attrs:
                    if isinstance(attr, dict) and attr.get('marketplace_id') == self.config['sp_api']['marketplace_id']:
                        if 'value' in attr:
                            result['メーカー名'] = attr['value']
                            break
                            
                # カテゴリ情報
                if item.get('productTypes'):
                    for product_type in item.get('productTypes', []):
                        if product_type.get('marketplaceId') == self.config['sp_api']['marketplace_id']:
                            result['カテゴリー'] = product_type.get('productType', '')
                            break
            except Exception as e:
                logging.error(f"追加情報の解析エラー: {str(e)}")
            
        except Exception as e:
            logging.error(f"カタログデータの解析エラー: {str(e)}")
            logging.error(traceback.format_exc())
        
        return result

In [19]:
# セル8: 価格情報と出品者情報取得
class AmazonProductAnalyzer(AmazonProductAnalyzer):
# ここから最新プライシングAPI
    def get_pricing_data_batch_v2(self, asins: list, batch_size: int = 20) -> list:
        """
        商品価格設定API v2022-05-01を使用してバッチ処理で複数ASINの価格情報を一度に取得する
        
        Parameters:
        -----------
        asins : list
            処理するASINのリスト
        batch_size : int
            1回のAPIリクエストで処理するASIN数（最大20）
            
        Returns:
        --------
        list: 商品価格情報のリスト
        """
        # トークンの更新チェック
        self.refresh_token_if_needed()
        
        # バッチサイズの上限を20に制限（API制限）
        batch_size = min(batch_size, 20)
        
        results = []
        
        # ASINをバッチに分割
        asin_batches = [asins[i:i+batch_size] for i in range(0, len(asins), batch_size)]
        
        for batch_idx, batch in enumerate(asin_batches, 1):
            print(f"Pricing APIバッチ処理中: {batch_idx}/{len(asin_batches)} ({len(batch)}件)")
            
            # このバッチが成功するまで繰り返す
            retry_count = 0
            max_retries = 5  # 最大再試行回数
            batch_success = False
            
            while not batch_success and retry_count < max_retries:
                try:
                    # バッチリクエストの構築
                    requests_data = []
                    for asin in batch:
                        requests_data.append({
                            "asin": asin,
                            "marketplaceId": self.config['sp_api']['marketplace_id'],
                            "includedData": [
                                "featuredBuyingOptions",
                                "referencePrices",
                                "lowestPricedOffers"
                            ],
                            "lowestPricedOffersInputs": [{
                                "itemCondition": "New",
                                "offerType": "Consumer"
                            }],
                            "uri": "/products/pricing/2022-05-01/items/competitiveSummary",
                            "method": "GET"
                        })
                    
                    # API エンドポイントとヘッダーの設定
                    url = 'https://sellingpartnerapi-fe.amazon.com/batches/products/pricing/2022-05-01/items/competitiveSummary'
                    headers = {
                        'x-amz-access-token': self.access_token,
                        'Content-Type': 'application/json',
                        'Accept': 'application/json'
                    }
                    
                    # バッチリクエストの送信
                    response = requests.post(url, headers=headers, json={"requests": requests_data})
                    
                    # レート制限の場合は待機して再試行(30.3秒待機の根拠は1秒あたりのレート制限：0.033req/1sから逆算　※待機時間 = 1/0.033 = 30.3秒)
                    if response.status_code == 429:
                        wait_time = int(response.headers.get('Retry-After', 31))
                        logging.warning(f"レート制限に達しました。{wait_time}秒待機して再試行します... (試行 {retry_count+1}/{max_retries})")
                        print(f"⚠️ レート制限に達しました。{wait_time}秒待機して再試行します...")
                        time.sleep(wait_time)
                        retry_count += 1
                        continue
                        
                    # その他のエラー処理
                    if response.status_code != 200:
                        logging.error(f"Pricing API バッチリクエストエラー: {response.status_code} - {response.text}")
                        # トークン期限切れの場合はトークンを更新して再試行
                        if response.status_code == 403:
                            response_text = response.text
                            if "expired" in response_text or "Unauthorized" in response_text:
                                logging.warning("トークン期限切れを検出。トークンを更新します。")
                                self.access_token = self.get_access_token()
                                self.token_timestamp = time.time()
                                headers['x-amz-access-token'] = self.access_token  # ヘッダーを更新
                                retry_count += 1
                                continue
                        
                        # その他のエラーは最大再試行回数まで試す
                        retry_count += 1
                        time.sleep(2)  # エラー時の待機
                        continue
                    
                    # 成功した場合の処理
                    response_data = response.json()
                    batch_results = self.parse_pricing_batch_response(response_data, batch)
                    results.extend(batch_results)
                    
                    # このバッチは成功
                    batch_success = True
                    
                except Exception as e:
                    logging.error(f"Pricing APIバッチ処理エラー: {str(e)}")
                    logging.error(traceback.format_exc())
                    retry_count += 1
                    if retry_count < max_retries:
                        time.sleep(2)  # 例外発生時の待機
                    
            # バッチが最大再試行回数を超えても成功しなかった場合
            if not batch_success:
                logging.error(f"バッチ {batch_idx}/{len(asin_batches)} は最大再試行回数を超えても成功しませんでした")
                print(f"❌ バッチ {batch_idx}/{len(asin_batches)} の処理に失敗しました")
            
            # バッチ間の待機（レート制限対策）- 最後のバッチ以外
            if batch_idx < len(asin_batches):
                batch_wait_time = self.config['sp_api'].get('batch_delay', 10.0)
                print(f"{batch_wait_time}秒間待機中...")
                time.sleep(batch_wait_time)
        
        return results
    
    
    def parse_pricing_batch_response(self, response_data, asins):
        """
        バッチ処理のレスポンスから価格情報を解析する
        
        Parameters:
        -----------
        response_data : dict
            APIレスポンスデータ
        asins : list
            リクエストに含まれていたASINのリスト（順序の対応付けのため）
            
        Returns:
        --------
        list: 解析された価格情報のリスト
        """
        results = []
        
        # レスポンスに "responses" キーがあることを確認
        if "responses" not in response_data:
            logging.error("Pricing APIレスポンスに 'responses' キーがありません")
            return results
            
        responses = response_data["responses"]
        
        # ASINとレスポンスの対応付け
        asin_to_response = {}
        for i, response in enumerate(responses):
            if i < len(asins):
                asin_to_response[asins[i]] = response
            else:
                logging.warning(f"レスポンス数がASIN数を超えています: {len(responses)} > {len(asins)}")
        
        # 各ASINに対する結果を処理
        for asin in asins:
            result = {
                'ASIN': asin,
                'Amazon価格': None,
                'カート価格': None,
                'カート価格送料': None,
                'カート価格のポイント': None,
                'カートセラーID': None,  # 追加
                # 'リードタイム（時間）': None,  # 削除
                'FBA最安値': None,
                'FBA最安値のポイント': None,
                '自己発送最安値': None,
                '自己発送最安値の送料': None,
                '自己発送最安値のポイント': None,
                'Amazon本体有無1': False,
                'FBA数': 0,
                '自己発送数': 0,
                '新品総出品者数': 0,
                'FBA最安値出品者数': 0,
                '自己発送最安値出品者数': 0
            }
            
            # 対応するレスポンスが存在するか確認
            if asin not in asin_to_response:
                logging.warning(f"ASIN {asin} に対応するレスポンスが見つかりません")
                results.append(result)
                continue
                
            response = asin_to_response[asin]
            
            # レスポンスにエラーがないか確認
            if "statusCode" in response and response["statusCode"] != 200:
                logging.warning(f"ASIN {asin} のレスポンスにエラー: {response.get('body', {}).get('errors', [])})")
                results.append(result)
                continue
                
            # レスポンスボディを取得
            if "body" not in response:
                logging.warning(f"ASIN {asin} のレスポンスに 'body' キーがありません")
                results.append(result)
                continue
                
            body = response["body"]
            
            # featuredBuyingOptions（カート価格情報）の処理
            if "featuredBuyingOptions" in body:
                featured_options = body["featuredBuyingOptions"]
                for option in featured_options:
                    # 新品のみを対象
                    if option.get("buyingOptionType") == "New":
                        # segmentedFeaturedOffers から情報を取得
                        segmented_offers = option.get("segmentedFeaturedOffers", [])
                        if segmented_offers:
                            # 最初のオファーを使用（通常はカートボックス）
                            featured_offer = segmented_offers[0]
                            
                            # カートセラーID
                            result["カートセラーID"] = featured_offer.get("sellerId", "")
                            
                            # カート価格
                            listing_price = featured_offer.get("listingPrice", {})
                            result["カート価格"] = listing_price.get("amount", None)
                            
                            # 送料情報
                            shipping_options = featured_offer.get("shippingOptions", [])
                            if shipping_options:
                                # デフォルトの送料を探す
                                for ship_option in shipping_options:
                                    if ship_option.get("shippingOptionType") == "DEFAULT":
                                        result["カート価格送料"] = ship_option.get("price", {}).get("amount", 0)
                                        break
                            
                            # ポイント情報
                            points = featured_offer.get("points", {})
                            result["カート価格のポイント"] = points.get("pointsNumber", 0)
                            
                            # リードタイム（時間）列は削除するため、ここでは設定しない
                            break
            
            # lowestPricedOffers（最安値情報）の処理
            if "lowestPricedOffers" in body:
                offers_list = body["lowestPricedOffers"]
                
                # FBA出品とマーチャント出品をグループ化
                fba_offers = []
                merchant_offers = []
                
                # レスポンスからlowestPricedOffersを抽出（リスト形式）
                for offer_group in offers_list:
                    # "New"条件の商品だけを処理
                    if "lowestPricedOffersInput" in offer_group and offer_group["lowestPricedOffersInput"]["itemCondition"] == "New":
                        condition_offers = offer_group.get("offers", [])
                        for offer in condition_offers:
                            # 販売者情報
                            seller_info = offer.get("seller", {})
                            # 提供されたJSONではseller_idフィールドが見つからないため
                            # sellerIdを直接使用
                            seller_id = offer.get("sellerId", "")
                            is_amazon = seller_id == "AN1VRQENFRJN5"  # Amazonの販売者ID
                            
                            # 配送タイプの確認（提供されたJSONに合わせて）
                            is_fba = offer.get("fulfillmentType") == "AFN"
                            
                            # 価格情報（提供されたJSONに合わせて）
                            price_info = offer.get("listingPrice", {})
                            offer_price = price_info.get("amount", 0)
                            
                            # 送料情報（提供されたJSONに合わせて構造を修正）
                            shipping_price = 0
                            shipping_options = offer.get("shippingOptions", [])
                            if shipping_options:
                                # デフォルトの送料を探す
                                for option in shipping_options:
                                    if option.get("shippingOptionType") == "DEFAULT":
                                        shipping_price = option.get("price", {}).get("amount", 0)
                                        break
                            
                            # ポイント情報（提供されたJSONに合わせて）
                            points = offer.get("points", {})
                            points_value = points.get("pointsNumber", 0)
                            
                            # Amazon情報
                            if is_amazon:
                                result["Amazon本体有無1"] = True
                                result["Amazon価格"] = offer_price
                            
                            # FBA/自己発送の分類
                            if is_fba:
                                result["FBA数"] += 1
                                fba_offers.append({
                                    "price": offer_price,
                                    "points": points_value,
                                    "seller_id": seller_id
                                })
                            else:
                                result["自己発送数"] += 1
                                merchant_offers.append({
                                    "price": offer_price,
                                    "shipping": shipping_price,
                                    "points": points_value,
                                    "seller_id": seller_id
                                })
                
                # 出品者数の合計
                result["新品総出品者数"] = result["FBA数"] + result["自己発送数"]
                
                # FBA最安値の計算
                if fba_offers:
                    min_fba_price = min(offer["price"] for offer in fba_offers)
                    min_fba_offers = [offer for offer in fba_offers if offer["price"] == min_fba_price]
                    result["FBA最安値"] = min_fba_price
                    result["FBA最安値のポイント"] = min_fba_offers[0]["points"]
                    result["FBA最安値出品者数"] = len(min_fba_offers)
                
                # 自己発送最安値の計算
                if merchant_offers:
                    min_merchant_total = min(offer["price"] + offer["shipping"] for offer in merchant_offers)
                    min_merchant_offers = [offer for offer in merchant_offers if offer["price"] + offer["shipping"] == min_merchant_total]
                    result["自己発送最安値"] = min_merchant_offers[0]["price"]
                    result["自己発送最安値の送料"] = min_merchant_offers[0]["shipping"]
                    result["自己発送最安値のポイント"] = min_merchant_offers[0]["points"]
                    result["自己発送最安値出品者数"] = len(min_merchant_offers)
            
            results.append(result)
        
        return results

In [21]:
# セル9: CSVファイル処理と結果保存
class AmazonProductAnalyzer(AmazonProductAnalyzer):                   
    def _save_results(self, results: List[Dict], output_file: str) -> None:
        """結果をCSVファイルに保存する"""
        try:
            if not results:
                logging.warning(f"保存する結果がありません: {output_file}")
                return
                    
            # 出力パスが相対パスの場合、dataディレクトリを基準にする
            if not os.path.isabs(output_file):
                output_file = os.path.join(self.data_dir, output_file)
                    
            # 出力ディレクトリの確認
            output_dir = os.path.dirname(output_file)
            if output_dir and not os.path.exists(output_dir):
                os.makedirs(output_dir)
            
            # すべての結果から一意のフィールド名を収集する
            all_fields = set()
            for row in results:
                all_fields.update(row.keys())
            
            # フィールド名の順序を決定
            # 初期フィールド順序（優先順位の高いフィールド）
            priority_fields = [
                "ASIN", "JAN", "商品名", "カテゴリー", "メーカー型番", "レビュー有無", 
                "メーカー名", "ブランド名", "総出品者数", "セット数", "商品追跡日", 
                "商品発売日", "追跡開始からの経過日数", "アダルト商品対象", "画像URL",
                
                # ランキング・URL情報
                "30日間平均ランキング", "90日間平均ランキング", "180日間平均ランキング",
                "amazonURL", "KeepaURL", "バリエーションASIN",
                
                # 参考価格、パッケージ情報
                "参考価格", "パッケージ最長辺", "パッケージ中辺", "パッケージ最短辺", "パッケージ重量",
                "現在ランキング",
                
                # 価格情報
                "Amazon価格", "カート価格", "カート価格送料", "カート価格のポイント", "リードタイム（時間）",
                "FBA最安値", "FBA最安値のポイント", "自己発送最安値", "自己発送最安値の送料", "自己発送最安値のポイント",
                "FBA_販売手数料", "FBA_配送代行手数料",
                
                # Amazon情報、出品者情報
                "Amazon本体有無1", "FBA数", "自己発送数", "新品総出品者数", 
                "FBA最安値出品者数", "自己発送最安値出品者数",
                
                # その他の情報
                "元コード", "コードタイプ"
            ]
            
            # フィールド名リストを構築
            fieldnames = []
            # 1. まず優先フィールドを追加
            for field in priority_fields:
                if field in all_fields:
                    fieldnames.append(field)
                    all_fields.remove(field)
            
            # 2. 残りのフィールドを追加（アルファベット順）
            remaining_fields = sorted(list(all_fields))
            fieldnames.extend(remaining_fields)
            
            # 各行が同じフィールドセットを持つようにする
            normalized_results = []
            for row in results:
                normalized_row = {field: row.get(field, None) for field in fieldnames}
                
                # パッケージ重量を丸める
                if normalized_row.get('パッケージ重量') is not None:
                    normalized_row['パッケージ重量'] = round(normalized_row['パッケージ重量'], 2)
                    
                normalized_results.append(normalized_row)
            
            # 書き込み
            with open(output_file, 'w', encoding='utf-8', newline='') as f:
                writer = csv.DictWriter(f, fieldnames=fieldnames)
                writer.writeheader()
                for row in normalized_results:
                    writer.writerow(row)
                        
            logging.info(f"結果を保存しました: {output_file} ({len(results)}件)")
            
        except Exception as e:
            logging.error(f"結果の保存中にエラー: {str(e)}")
            logging.error(traceback.format_exc())
            raise

    def load_codes_from_file(self, input_file: str) -> tuple:
        """
        CSVファイルからコードを読み込む（重複を除去し、価格が低い方を優先）
        
        Parameters:
        -----------
        input_file : str
            入力CSVファイルのパス
            
        Returns:
        --------
        tuple: (codes, duplicates_count)
            codes: 処理対象のコードリスト
            duplicates_count: 除外された重複の数
        """
        try:
            if not os.path.exists(input_file):
                raise FileNotFoundError(f"入力ファイルが見つかりません: {input_file}")
                
            # CSVファイルの読み込み
            with open(input_file, 'r', encoding='utf-8') as f:
                reader = csv.DictReader(f)
                
                # コード列の特定
                code_column = None
                for name in reader.fieldnames:
                    if name.upper() in ['CODE', 'JAN', 'EAN', 'ASIN', 'PRODUCT_CODE', 'JANコード']:
                        code_column = name
                        break
                        
                # 価格列の特定
                price_column = None
                for name in reader.fieldnames:
                    if name in ['価格', 'PRICE', '仕入価格', '仕入れ価格', 'COST']:
                        price_column = name
                        break
                
                if not code_column:
                    raise ValueError("有効なコード列(CODE/JAN/EAN/ASIN)がCSVに見つかりません")
                
                # すべての行を読み込む
                all_rows = []
                for row in reader:
                    if row[code_column].strip():
                        # 価格列がない場合や価格が空の場合は無限大とする
                        price = float('inf')
                        if price_column and row[price_column].strip():
                            try:
                                price = float(row[price_column].strip().replace(',', ''))
                            except (ValueError, TypeError):
                                pass
                        
                        all_rows.append({
                            'code': row[code_column].strip(),
                            'price': price,
                            'original_row': row
                        })
            
            # 重複を除去し、価格が小さい方を残す
            code_price_dict = {}  # {コード: (価格, 行インデックス)}
            
            for i, row_data in enumerate(all_rows):
                code = row_data['code']
                price = row_data['price']
                
                if code in code_price_dict:
                    current_price, _ = code_price_dict[code]
                    if price < current_price:  # 価格が小さい方を優先
                        code_price_dict[code] = (price, i)
                else:
                    code_price_dict[code] = (price, i)
            
            # 価格が小さい方を残したユニークなコードリスト
            unique_codes = []
            for code, (_, index) in code_price_dict.items():
                unique_codes.append(all_rows[index]['code'])
            
            # 重複数の計算
            duplicates_count = len(all_rows) - len(unique_codes)
            
            logging.info(f"{len(unique_codes)}件のコードを読み込みました（重複除外: {duplicates_count}件）")
            return unique_codes, duplicates_count
            
        except Exception as e:
            logging.error(f"コード読み込みエラー: {str(e)}")
            raise

In [23]:
# セル10: フィルタリング機能
class AmazonProductAnalyzer(AmazonProductAnalyzer):
    def filter_product(self, product_data: Dict) -> Optional[Dict]:
        """商品データのフィルタリング
        
        設定ファイルのfiltersセクションに基づいて商品データをフィルタリングする。
        各フィルター条件はNoneの場合、その条件はスキップされる。
        """
        try:
            filters = self.config['filters']
            conditions = {}
            
            # 条件設定（基本的にfiltersに各条件があれば追加）
            
            # カート価格条件（min/maxが設定されている場合のみ追加）
            if 'price' in filters:
                price_min = filters['price'].get('min')
                price_max = filters['price'].get('max')
                
                # 価格条件：minとmaxの両方、またはどちらか一方が設定されている場合
                if price_min is not None or price_max is not None:
                    conditions['価格範囲'] = lambda p: (
                        (price_min is None or (p['カート価格'] is not None and price_min <= p['カート価格'])) and
                        (price_max is None or (p['カート価格'] is not None and p['カート価格'] <= price_max))
                    )
            
            # ランキング条件（min/maxが設定されている場合のみ追加）
            if 'ranking' in filters:
                rank_min = filters['ranking'].get('min')
                rank_max = filters['ranking'].get('max')
                
                # ランキング条件：minとmaxの両方、またはどちらか一方が設定されている場合
                if rank_min is not None or rank_max is not None:
                    conditions['ランキング'] = lambda p: (
                        (rank_min is None or (p['現在ランキング'] is not None and rank_min <= p['現在ランキング'])) and
                        (rank_max is None or (p['現在ランキング'] is not None and p['現在ランキング'] <= rank_max))
                    )
            
            # Amazonの出品有無条件
            conditions['Amazon出品なし'] = lambda p: not p['Amazon本体有無1']
            
            # 出品者数条件
            if 'sellers' in filters and 'total' in filters['sellers']:
                total_min = filters['sellers']['total'].get('min')
                total_max = filters['sellers']['total'].get('max')
                
                if total_min is not None or total_max is not None:
                    conditions['出品者数'] = lambda p: (
                        (total_min is None or total_min <= p['新品総出品者数']) and
                        (total_max is None or p['新品総出品者数'] <= total_max)
                    )
            
            # FBA出品者数条件
            if 'sellers' in filters and 'fba' in filters['sellers']:
                fba_min = filters['sellers']['fba'].get('min')
                fba_max = filters['sellers']['fba'].get('max')
                
                if fba_min is not None or fba_max is not None:
                    conditions['FBA出品者数'] = lambda p: (
                        (fba_min is None or fba_min <= p['FBA数']) and
                        (fba_max is None or p['FBA数'] <= fba_max)
                    )
    
            # 各条件をチェック（いずれかの条件に合致しなければ除外）
            for condition_name, check in conditions.items():
                if not check(product_data):
                    logging.info(f"フィルター除外 - {condition_name}: {product_data['ASIN']}")
                    return None
    
            # すべての条件を満たした場合は商品データを返す
            return product_data
    
        except Exception as e:
            logging.error(f"フィルタリング中にエラー: {str(e)}")
            logging.error(traceback.format_exc())
            return None

    def filter_products(self, products: list) -> list:
        """
        商品リストをフィルタリングする
        
        Parameters:
        -----------
        products : list
            フィルタリング対象の商品リスト
            
        Returns:
        --------
        list: フィルタリング後の商品リスト
        """
        filtered_products = []
        
        for product in products:
            filtered_product = self.filter_product(product)
            if filtered_product:
                filtered_products.append(filtered_product)
        
        logging.info(f"フィルタリング: {len(products)}件中{len(filtered_products)}件が条件を満たしました")
        return filtered_products

In [25]:
# セル11: 統合バッチ処理
class AmazonProductAnalyzer(AmazonProductAnalyzer):
    def get_pricing_data_batch(self, catalog_data: list, batch_size: int = 5) -> list:
        """
        バッチ処理でPricing APIを使用して複数ASINの価格情報を取得し、Catalog情報と結合
        
        Parameters:
        -----------
        catalog_data : list
            Catalog APIで取得した商品基本情報のリスト
        batch_size : int
            バッチサイズ
                
        Returns:
        --------
        list: 完全な商品情報のリスト
        """
        # 処理対象ASINリスト
        asins_to_process = [item['ASIN'] for item in catalog_data]
        
        print(f"Pricing API処理対象: {len(asins_to_process)}件")
        
        # 新しいバッチAPIを使用して価格情報を取得
        pricing_results = self.get_pricing_data_batch_v2(asins_to_process, batch_size)
        
        # 結果をカタログデータと結合
        complete_results = []
        
        # 各カタログデータに対応する価格情報を探して結合
        for item in catalog_data:
            asin = item['ASIN']
            
            # 対応する価格情報を探す
            pricing_data = next((p for p in pricing_results if p['ASIN'] == asin), None)
            
            if pricing_data:
                # カタログデータと価格情報を結合
                result = {**item, **pricing_data}
                complete_results.append(result)
                print(f"✅ {asin}: 商品情報の結合に成功")
            else:
                # 価格情報がない場合はカタログデータのみ使用
                print(f"⚠️ {asin}: 価格情報がありません")
                complete_results.append(item)
        
        logging.info(f"商品情報の結合完了: {len(complete_results)}/{len(catalog_data)}件")
        return complete_results

    
    def process_codes_and_get_catalog_data(self, codes: list, batch_size: int = 50, max_ranking: int = 100000) -> list:
        """
        コード変換とCatalog情報取得を一度のAPI呼び出しで行う統合メソッド
        
        Parameters:
        -----------
        codes : list
            処理するコードのリスト（JANまたはASIN）
        batch_size : int
            バッチサイズ（デフォルト: 50）
        max_ranking : int
            処理する最大ランキング値（これより大きいランキングの商品は除外）
            
        Returns:
        --------
        list: カタログ情報を含む商品データのリスト（ランキングフィルター適用済み）
        """
        catalog_results = []
        filtered_results = []  # ランキングでフィルターした結果を格納
        
        # コードのバッチ処理
        code_batches = [codes[i:i+batch_size] for i in range(0, len(codes), batch_size)]
        
        for batch_idx, batch in enumerate(code_batches, 1):
            print(f"コード処理バッチ: {batch_idx}/{len(code_batches)} ({len(batch)}件)")
            
            # トークンの更新チェック
            self.refresh_token_if_needed()
            
            # バッチ内の各コードを処理
            batch_results = []
            for code in batch:
                try:
                    # コードタイプの判定 - タプルを返すように変更
                    code_type, normalized_code = self.identify_code_type(code)
                    
                    if code_type == 'EAN':
                        print(f"JAN/EAN: {code}の処理中...")
                        # JAN/EANコードでCatalog API v2022-04-01を呼び出す
                        item_data = self.get_item_data_by_code(normalized_code, 'EAN')
                        
                        if not item_data:
                            print(f"⚠️ JAN/EAN {code} の商品情報が見つかりません")
                            continue
                            
                        # ASINを取得
                        asin = item_data.get('asin')
                        if not asin:
                            print(f"⚠️ JAN/EAN {code} に対応するASINが見つかりません")
                            continue
                            
                        print(f"✓ JAN/EAN {code} → ASIN: {asin} 変換成功")
                            
                    else:
                        # 既にASINの場合は直接Catalog API v2022-04-01を呼び出す
                        print(f"ASIN: {code}の処理中...")
                        asin = normalized_code
                        item_data = self.get_item_data_by_code(asin, 'ASIN')
                        
                        if not item_data:
                            print(f"⚠️ ASIN {asin} の商品情報が見つかりません")
                            continue
                    
                    # カタログデータを解析してフォーマット
                    catalog_data = self.parse_catalog_data(item_data)
                    
                    # 基本情報を追加
                    result = {
                        'ASIN': asin,
                        '元コード': code,
                        'コードタイプ': code_type,
                        # catalog_dataの内容をマージ
                        **catalog_data
                    }
                    
                    batch_results.append(result)
                    
                    # ランキングが指定の値以下かチェック
                    ranking = result.get('現在ランキング')
                    if ranking is not None and ranking <= max_ranking:
                        filtered_results.append(result)
                        print(f"✅ {code}: 商品情報取得成功 (ランキング: {ranking})")
                    else:
                        print(f"⏭️ {code}: ランキング条件を満たさないため除外 (ランキング: {ranking})")
                    
                except Exception as e:
                    print(f"❌ {code}: 処理エラー - {str(e)}")
                    logging.error(f"コード処理エラー ({code}): {str(e)}")
                    logging.error(traceback.format_exc())
            
            # 結果を追加
            catalog_results.extend(batch_results)
        
        total_success = len(catalog_results)
        total_filtered = len(filtered_results)
        total_codes = len(codes)
        logging.info(f"コード処理とカタログ情報取得完了: {total_success}/{total_codes}件 (成功率: {total_success/total_codes*100:.1f}%)")
    
        # この部分を修正
        percentage = (total_filtered/total_success*100) if total_success > 0 else 0
        logging.info(f"ランキングフィルター適用後: {total_filtered}/{total_success}件 (通過率: {percentage:.1f}%)")
        
        return filtered_results  # ランキングフィルター適用後の結果を返す


In [27]:
# セル12: 実行用コード
def test_improved_batch_processing():
    try:
        # アナライザーのインスタンス作成
        analyzer = AmazonProductAnalyzer()
        
        # 設定ファイルからデフォルト値を読み込む
        input_file = None  # Noneを指定すると設定ファイルの値が使用される
        output_file = None  # Noneを指定すると設定ファイルの値が使用される
        batch_size = 20  # テスト用に小さいバッチサイズ
        max_ranking = 100000  # ランキング条件（10万位以下の商品のみ処理）
        
        print(f"改善されたバッチ処理テスト開始 (バッチサイズ: {batch_size}, 最大ランキング: {max_ranking})")
        print(f"入力ファイル: {input_file if input_file else analyzer.config['sp_api']['output']['input_file']}")
        print(f"出力ファイル: {output_file if output_file else analyzer.config['sp_api']['output']['output_file']}")
        
        # 入出力ファイルのパスを設定
        if input_file is None:
            input_file = analyzer.config['sp_api']['output']['input_file']
        
        if output_file is None:
            output_file = analyzer.config['sp_api']['output']['output_file']
        
        # 実行時間計測開始
        start_time = time.time()
        
        # CSVファイルからコードを読み込む
        codes, duplicates_count = analyzer.load_codes_from_file(input_file)
        total_codes = len(codes)
        
        if duplicates_count > 0:
            print(f"ℹ️ 入力ファイル内の重複: {duplicates_count}件をスキップしました")
        
        print(f"\n全{total_codes}件の処理を開始します...")
        
        # ステップ1+2: コード変換とCatalog情報取得を一度に実行（統合）- ランキングフィルター付き
        print(f"\n📌 ステップ1+2: コード変換とCatalog情報取得を統合処理（最大ランキング: {max_ranking}）")
        catalog_data = analyzer.process_codes_and_get_catalog_data(codes, batch_size, max_ranking)
        print(f"✅ 統合処理完了: {len(catalog_data)}/{total_codes}件の商品情報を取得（ランキング条件適用済み）")
        
        if not catalog_data:
            print("\n⚠️ ランキング条件を満たす商品がありませんでした。処理を終了します。")
            return
        
        # ステップ3: Pricing APIで価格情報を取得して結合
        print("\n📌 ステップ3: Pricing APIで価格情報を取得して結合")
        complete_data = analyzer.get_pricing_data_batch(catalog_data, batch_size)
        print(f"✅ Pricing API処理完了: {len(complete_data)}/{len(catalog_data)}件の商品情報を取得")
        
        # ステップ4: フィルタリングと保存
        print("\n📌 ステップ4: フィルタリングと結果の保存")
        filtered_data = analyzer.filter_products(complete_data)
        
        # 結果の保存
        if complete_data:
            analyzer._save_results(complete_data, output_file)
            print(f"全商品データを保存しました: {output_file} ({len(complete_data)}件)")
            
            filtered_output = output_file.replace('.csv', '_filtered.csv')
            if filtered_data:
                analyzer._save_results(filtered_data, filtered_output)
                print(f"フィルタリング後のデータを保存しました: {filtered_output} ({len(filtered_data)}件)")
        
        # 処理時間の表示
        end_time = time.time()
        elapsed = end_time - start_time
        print(f"\n処理完了！実行時間: {elapsed:.2f}秒")
        
        # サマリーの表示
        print("\n==== 処理結果サマリー ====")
        print(f"総処理件数: {total_codes}")
        print(f"ランキング条件通過: {len(catalog_data)}")
        print(f"価格情報取得成功: {len(complete_data)}")
        print(f"最終フィルタリング後: {len(filtered_data)}")
        
    except Exception as e:
        print(f"エラーが発生しました: {str(e)}")
        import traceback
        print(traceback.format_exc())


# # テスト実行
test_improved_batch_processing()

2025-03-29 12:20:22,372 - WARNING - ログ機能の初期化が完了しました: C:\Users\inato\Documents\amazon-research\logs\sp_api_20250329_122022.log


ログファイル: C:\Users\inato\Documents\amazon-research\logs\sp_api_20250329_122022.log
改善されたバッチ処理テスト開始 (バッチサイズ: 20, 最大ランキング: 100000)
入力ファイル: C:\Users\inato\Documents\amazon-research\data\netsea_scraping.csv
出力ファイル: C:\Users\inato\Documents\amazon-research\data\sp_api_output.csv
ℹ️ 入力ファイル内の重複: 16件をスキップしました

全300件の処理を開始します...

📌 ステップ1+2: コード変換とCatalog情報取得を統合処理（最大ランキング: 100000）
コード処理バッチ: 1/15 (20件)
JAN/EAN: 4980901211667の処理中...
✓ JAN/EAN 4980901211667 → ASIN: B08ZMQ3MPC 変換成功
⏭️ 4980901211667: ランキング条件を満たさないため除外 (ランキング: 118084)
JAN/EAN: 4901933040391の処理中...
✓ JAN/EAN 4901933040391 → ASIN: B000FQNSSU 変換成功
✅ 4901933040391: 商品情報取得成功 (ランキング: 1537)
JAN/EAN: 4974234619399の処理中...
✓ JAN/EAN 4974234619399 → ASIN: B0DLBB3PMX 変換成功
✅ 4974234619399: 商品情報取得成功 (ランキング: 293)
JAN/EAN: 4975416827052の処理中...
✓ JAN/EAN 4975416827052 → ASIN: B000FQSODY 変換成功
✅ 4975416827052: 商品情報取得成功 (ランキング: 269)
JAN/EAN: 4902522675802の処理中...
✓ JAN/EAN 4902522675802 → ASIN: B07TWR68D4 変換成功
✅ 4902522675802: 商品情報取得成功 (ランキング: 4651)
JAN/EAN

2025-03-29 12:20:46,494 - WARNING - EAN 4511413408049 の商品情報が見つかりませんでした


⚠️ JAN/EAN 4511413408049 の商品情報が見つかりません
JAN/EAN: 4903111503629の処理中...
✓ JAN/EAN 4903111503629 → ASIN: B09DSVG7KQ 変換成功
✅ 4903111503629: 商品情報取得成功 (ランキング: 7071)
コード処理バッチ: 3/15 (20件)
JAN/EAN: 4517739006730の処理中...
✓ JAN/EAN 4517739006730 → ASIN: B0BVRGBD4R 変換成功
✅ 4517739006730: 商品情報取得成功 (ランキング: 12760)
JAN/EAN: 4973512280405の処理中...
✓ JAN/EAN 4973512280405 → ASIN: B0CXT1W2H2 変換成功
✅ 4973512280405: 商品情報取得成功 (ランキング: 10203)
JAN/EAN: 4973512280399の処理中...
✓ JAN/EAN 4973512280399 → ASIN: B0CXT7N6MK 変換成功
✅ 4973512280399: 商品情報取得成功 (ランキング: 5740)
JAN/EAN: 4903111319374の処理中...
✓ JAN/EAN 4903111319374 → ASIN: B0C43CBNQJ 変換成功
⏭️ 4903111319374: ランキング条件を満たさないため除外 (ランキング: 208905)
JAN/EAN: 4903111549450の処理中...
✓ JAN/EAN 4903111549450 → ASIN: B0C9Y83HSG 変換成功
⏭️ 4903111549450: ランキング条件を満たさないため除外 (ランキング: 103192)
JAN/EAN: 4976558007630の処理中...
✓ JAN/EAN 4976558007630 → ASIN: B0DFH2CXSM 変換成功
✅ 4976558007630: 商品情報取得成功 (ランキング: 10636)
JAN/EAN: 4904140583149の処理中...
✓ JAN/EAN 4904140583149 → ASIN: B00QLFCAWO 変換成功
✅ 4904140

2025-03-29 12:21:14,791 - WARNING - EAN 4562191602860 の商品情報が見つかりませんでした


...

⚠️ レート制限に達しました。31秒待機して再試行します...
10.0秒間待機中...
Pricing APIバッチ処理中: 9/12 (20件)
10.0秒間待機中...
Pricing APIバッチ処理中: 10/12 (20件)


2025-03-29 12:27:48,679 - WARNING - レート制限に達しました。31秒待機して再試行します... (試行 1/5)


⚠️ レート制限に達しました。31秒待機して再試行します...
10.0秒間待機中...
Pricing APIバッチ処理中: 11/12 (20件)


2025-03-29 12:28:30,865 - WARNING - レート制限に達しました。31秒待機して再試行します... (試行 1/5)


⚠️ レート制限に達しました。31秒待機して再試行します...
10.0秒間待機中...
Pricing APIバッチ処理中: 12/12 (14件)
✅ B000FQNSSU: 商品情報の結合に成功
✅ B0DLBB3PMX: 商品情報の結合に成功
✅ B000FQSODY: 商品情報の結合に成功
✅ B07TWR68D4: 商品情報の結合に成功
✅ B0DL5JNN7K: 商品情報の結合に成功
✅ B0012VQMUS: 商品情報の結合に成功
✅ B00INJDWPK: 商品情報の結合に成功
✅ B003Y8Y9QG: 商品情報の結合に成功
✅ B0CVGLQ5M1: 商品情報の結合に成功
✅ B0BMYYL4Z2: 商品情報の結合に成功
✅ B00BIIALVQ: 商品情報の結合に成功
✅ B08FM1X43T: 商品情報の結合に成功
✅ B0152Z7T3I: 商品情報の結合に成功
✅ B09JKL4B8N: 商品情報の結合に成功
✅ B0CZP6427Z: 商品情報の結合に成功
✅ B07GYWS64P: 商品情報の結合に成功
✅ B000FQSODO: 商品情報の結合に成功
✅ B0C1MDN5VC: 商品情報の結合に成功
✅ B0CXSZ1RJV: 商品情報の結合に成功
✅ B08FM2WY4J: 商品情報の結合に成功
✅ B00BII8NCK: 商品情報の結合に成功
✅ B00F4LIET4: 商品情報の結合に成功
✅ B001J8YLR0: 商品情報の結合に成功
✅ B0CVKMG3ND: 商品情報の結合に成功
✅ B0DLFM4CQJ: 商品情報の結合に成功
✅ B0BSFCSYRX: 商品情報の結合に成功
✅ B01ICRZANO: 商品情報の結合に成功
✅ B01KNWP1ZI: 商品情報の結合に成功
✅ B009PI1HGY: 商品情報の結合に成功
✅ B09QC6JBZK: 商品情報の結合に成功
✅ B0BDF7DD27: 商品情報の結合に成功
✅ B0D47DZPDN: 商品情報の結合に成功
✅ B09DSVG7KQ: 商品情報の結合に成功
✅ B0BVRGBD4R: 商品情報の結合に成功
✅ B0CXT1W2H2: 商品情報の結合に成功
✅ B0CXT7N6MK: 商品情報の結合に成功
✅ B0DFH2CXSM: 商品情報の結合に成功